In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Paramètres de simulation
num_grains = 1000
space_size = 100
time_steps = 500
collision_distance = 1.0
dt = 0.01  # Petit pas de temps pour une meilleure résolution

# Fonction de simulation unique
def simulate_grains(angular_velocity, pressure_gradient, G, central_mass, num_grains, space_size, time_steps, dt):
    grains = np.zeros((num_grains, 7))  # x, y, vx, vy, masse, actif, taille initiale
    angles = np.linspace(0, 2 * np.pi, num_grains)
    radii = np.random.rand(num_grains) * space_size / 2 + space_size / 4
    grains[:, 0] = radii * np.cos(angles)
    grains[:, 1] = radii * np.sin(angles)
    grains[:, 2] = -angular_velocity * grains[:, 1]
    grains[:, 3] = angular_velocity * grains[:, 0]
    grains[:, 4] = 1  # Masse initiale
    grains[:, 5] = 1  # Actif

    initial_positions_x = grains[:, 0].copy()
    initial_positions_y = grains[:, 1].copy()

    trajectories_x = [[] for _ in range(num_grains)]
    trajectories_y = [[] for _ in range(num_grains)]

    def update_positions(grains, dt):
        for i in range(num_grains):
            if grains[i, 5] == 0:  # Si le grain n'est pas actif (fusionné), sauter
                continue

            pos = grains[i, :2]
            vel = grains[i, 2:4]

            centrifugal_force = np.array([-pos[1], pos[0]]) * angular_velocity ** 2
            pressure_force = -pos * pressure_gradient
            dist_to_star = np.linalg.norm(pos)
            gravity_force_star = -G * central_mass * pos / dist_to_star**3

            gravity_force_grains = np.zeros(2)
            for j in range(num_grains):
                if i != j and grains[j, 5] == 1:
                    dist = np.linalg.norm(grains[j, :2] - pos)
                    if dist > 0:  # Eviter la division par zéro
                        gravity_force_grains += G * grains[i, 4] * grains[j, 4] * (grains[j, :2] - pos) / dist**3

            total_force = centrifugal_force + pressure_force + gravity_force_star + gravity_force_grains

            grains[i, 2:4] += total_force / grains[i, 4] * dt
            grains[i, :2] += vel * dt

            trajectories_x[i].append(grains[i, 0])
            trajectories_y[i].append(grains[i, 1])

    def check_collisions(grains):
        for i in range(num_grains):
            if grains[i, 5] == 0:  # Si le grain n'est pas actif (fusionné), sauter
                continue
            for j in range(i + 1, num_grains):

                if grains[j, 5] == 0:  # Si le grain n'est pas actif (fusionné), sauter
                    continue
                dist = np.linalg.norm(grains[i, :2] - grains[j, :2])
                if dist < collision_distance:
                    total_mass = grains[i, 4] + grains[j, 4]
                    new_velocity = (grains[i, 2:4] * grains[i, 4] + grains[j, 2:4] * grains[j, 4]) / total_mass
                    grains[i, 2:4] = new_velocity
                    grains[i, 4] = total_mass
                    grains[j, 5] = 0

    for t in range(time_steps):
        update_positions(grains, dt)
        check_collisions(grains)

    num_active_grains = np.sum(grains[:, 5])
    return initial_positions_x, initial_positions_y, trajectories_x, trajectories_y, num_active_grains

# Paramètres par défaut
default_angular_velocity = 0.1
default_pressure_gradient = 0.1
default_G = 1.0
default_central_mass = 1000

# Valeurs à faire varier
variation_values = [0.05, 0.1, 0.15, 0.3]
variation_values_masses = [0.5 * default_central_mass, default_central_mass, 1.5 * default_central_mass, 3.0 * default_central_mass]

# Fonctions pour créer les subplots

def plot_simulations(param_name, values, fixed_params):
    fig, axs = plt.subplots(2, 2, figsize=(15, 12))
    axs = axs.flatten()
    
    for i, value in enumerate(values):
        if param_name == 'angular_velocity':
            params = {'angular_velocity': value, 'pressure_gradient': fixed_params['pressure_gradient'], 'G': fixed_params['G'], 'central_mass': fixed_params['central_mass']}
        elif param_name == 'pressure_gradient':
            params = {'angular_velocity': fixed_params['angular_velocity'], 'pressure_gradient': value, 'G': fixed_params['G'], 'central_mass': fixed_params['central_mass']}
        elif param_name == 'G':
            params = {'angular_velocity': fixed_params['angular_velocity'], 'pressure_gradient': fixed_params['pressure_gradient'], 'G': value, 'central_mass': fixed_params['central_mass']}
        elif param_name == 'central_mass':
            params = {'angular_velocity': fixed_params['angular_velocity'], 'pressure_gradient': fixed_params['pressure_gradient'], 'G': fixed_params['G'], 'central_mass': value}
        
        initial_positions_x, initial_positions_y, trajectories_x, trajectories_y, num_active_grains = simulate_grains(
            params['angular_velocity'], params['pressure_gradient'], params['G'], params['central_mass'],
            num_grains, space_size, time_steps, dt
        )

        axs[i].scatter(initial_positions_x, initial_positions_y, c='r', marker='+', label='Positions initiales')
        for j in range(num_grains):
            if len(trajectories_x[j]) > 0:
                axs[i].plot(trajectories_x[j], trajectories_y[j], alpha=0.5)

        axs[i].set_title(f'{param_name} = {value}')
        axs[i].set_xlim(-space_size, space_size)
        axs[i].set_ylim(-space_size, space_size)
        axs[i].set_xlabel('x')
        axs[i].set_ylabel('y')
        axs[i].legend()
        axs[i].text(0.05, 0.95, f'Grains finaux: {num_active_grains}', transform=axs[i].transAxes, fontsize=12,
                    verticalalignment='top')

    plt.suptitle(f'Variations de {param_name}')
    plt.tight_layout()
    plt.show()


# Créer des subplots pour chaque paramètre
fixed_params = {
    'angular_velocity': default_angular_velocity,
    'pressure_gradient': default_pressure_gradient,
    'G': default_G,
    'central_mass': default_central_mass
}
plot_simulations('angular_velocity', variation_values, fixed_params)
plot_simulations('pressure_gradient', variation_values, fixed_params)
plot_simulations('G', variation_values, fixed_params)
plot_simulations('central_mass', variation_values_masses, fixed_params)
